In [7]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.optimize as opt
import csv

from scipy.linalg import expm
from scipy.optimize import minimize

import matplotlib
matplotlib.rcParams['figure.dpi'] = 50

In [ ]:
#--- Static parameters and Hamiltonians ---
# Physical parameters (in MHz)
class PulseParameters:
    """
    Container for all static physical and derived pulse parameters, plus initial states and
    a time-grid generator.
    """
    def __init__(self,
                 Omega_Rabi1,
                 Omega_Rabi2,
                 blockade,
                 r_lifetime,
                 delta1,
                 delta2,
                 resolution
                 ):
        # raw parameters
        self.Omega_Rabi1 = Omega_Rabi1
        self.Omega_Rabi2 = Omega_Rabi2
        self.blockade    = blockade
        self.r_lifetime  = r_lifetime
        self.delta1      = delta1
        self.delta2      = delta2
        self.resolution  = resolution

        # normalized / derived
        self.normalized_Omega    = Omega_Rabi2 / Omega_Rabi1
        self.normalized_blockade = blockade / Omega_Rabi1
        self.decay_rate          = (1 / r_lifetime) / Omega_Rabi1

        # fixed initial states
        self.initial_psi01 = np.array([1, 0, 0], dtype=complex)
        self.initial_psi11 = np.array([1, 0, 0, 0, 0, 0, 0], dtype=complex)

    def time_grid(self, pulse_time):
        """
        Build a uniform time grid for a given pulse duration.
        Returns (times, dt).
        """
        times = np.linspace(0, pulse_time, self.resolution + 1)
        dt = times[1] - times[0]
        return times, dt


class Hamiltonians:
    """
    Provides Hamiltonian constructors given pulse parameters.
    """
    def __init__(self, params: PulseParameters):
        self.params = params

    def H11(self, phase_i):
        """Two-atom Hamiltonian at a given phase (Hermitian)."""
        Omega1 = np.exp(1j * phase_i)
        Omega2 = self.params.normalized_Omega * Omega1
        Delta1 = self.params.delta1
        Delta2 = self.params.delta2
        B  = self.params.normalized_blockade

        H = 0.5 * np.sqrt(2) * np.array([
            [0,               Omega1,            Omega2,        0,                   0,                   0,                   0],
            [np.conj(Omega1),     Delta1,            0,         Omega1,                 Omega2,                  0,                   0],
            [np.conj(Omega2),     0,             Delta2,        0,                   0,                   Omega1,                  Omega2],
            [0,               np.conj(Omega1),   0,         2*Delta1 + B,            0,                   0,                   0],
            [0,               np.conj(Omega2),   0,         0,                   Delta1 + Delta2 + B,          0,                   0],
            [0,               0,             np.conj(Omega1),0,                 0,                   Delta1 + Delta2 + B,          0],
            [0,               0,             np.conj(Omega2),0,                 0,                   0,                   2*Delta2 + B]
        ], complex)
        return H

    def H01(self, phase_i):
        """Single-atom Hamiltonian at a given phase (Hermitian)."""
        Omega1 = np.exp(1j * phase_i)
        Omega2 = self.params.normalized_Omega * Omega1
        Delta1 = self.params.delta1
        Delta2 = self.params.delta2

        H = 0.5 * np.array([
            [0,               Omega1,      Omega2],
            [np.conj(Omega1),     Delta1,      0 ],
            [np.conj(Omega2),     0,       Delta2]
        ], complex)
        return 0.5 * (H + H.conj().T)
    
    # Derivative of H11_control w.r.t. phase
    def dH11_dphi(self, phase_i):
        Omega1 = np.exp(1j * phase_i)
        Omega2 = self.params.normalized_Omega * Omega1
        dOmega1 = 1j * Omega1
        dOmega2 = 1j * Omega2
        dH = 0.5 * np.sqrt(2) * np.array([
            [0,               dOmega1,         dOmega2,       0,                    0,                    0,                    0],
            [np.conj(dOmega1),0,               0,             dOmega1,              dOmega2,              0,                    0],
            [np.conj(dOmega2),0,               0,             0,                    0,              dOmega1,              dOmega2],
            [0,               np.conj(dOmega1),0,             0,                    0,                    0,                    0],
            [0,               np.conj(dOmega2),0,             0,                    0,                    0,                    0],
            [0,               0,               np.conj(dOmega1),0,                  0,                    0,                    0],
            [0,               0,               np.conj(dOmega2),0,                  0,                    0,                    0]
        ], complex)
        return 0.5 * (dH + dH.conj().T)

    # Derivative of H01_control w.r.t. phase
    def dH01_dphi(self, phase_i):
        Omega1 = np.exp(1j * phase_i)
        Omega2 = self.params.normalized_Omega * Omega1
        dOmega1 = 1j * Omega1
        dOmega2 = 1j * Omega2
        dH = 0.5 * np.array([
            [0,               dOmega1,      dOmega2],
            [np.conj(dOmega1),0,            0     ],
            [np.conj(dOmega2),0,            0     ]
        ], complex)
        return 0.5 * (dH + dH.conj().T)

class FidelityCalculator:
    """
    Computes fidelities for given quantum states.
    """
    @staticmethod
    def bell_state_fidelity(psi01, psi11):
        """Compute Bell state fidelity from single- and two-atom amplitudes."""
        return (1/16) * np.abs(1 + 2 * psi01[0] - psi11[0])**2


In [ ]:
#--- CRAB ansatz and optimizer ---
class CosineAnsatz:
    """
    Parameterizes a phase profile as a sum of cosines plus a linear detuning term.
    """
    def __init__(self, num_terms: int):
        self.num_terms = num_terms

    def generate(self, params: PulseParameters, inputs: np.ndarray):
        """
        inputs layout: [pulse_time, drive_detuning, (Amp_i, freq_i, offset_i)*num_terms]
        Returns:
          phases: np.ndarray of length resolution
          dt: float timestep
        """
        pulse_time, drive_detuning = inputs[0], inputs[1]
        coeffs = inputs[2:]
        amps   = coeffs[0::3]
        freqs  = coeffs[1::3]
        offs   = coeffs[2::3]

        times, dt = params.time_grid(pulse_time)
        phases = np.zeros(params.resolution, dtype=float)
        for i, t in enumerate(times[:-1]):
            phi = (drive_detuning/params.Omega_Rabi1)*t
            for A, f, o in zip(amps, freqs, offs):
                phi += A * np.cos((t - pulse_time/2)*(f/params.Omega_Rabi1) - o)
            phases[i] = phi
        return phases, dt


class GateSimulator:
    """
    Evolves the two-level system under a given phase profile and returns fidelity.
    """
    def __init__(self,
                 params: PulseParameters,
                 hams: Hamiltonians,
                 fidelity_calc: FidelityCalculator
                 ):
        self.params = params
        self.hams   = hams
        self.fid    = fidelity_calc

    def run(self, phases: np.ndarray, dt: float) -> float:
        psi01 = self.params.initial_psi01.copy()
        psi11 = self.params.initial_psi11.copy()
        for phi in phases:
            U01 = scipy.linalg.expm(-1j*self.hams.H01(phi)*dt)
            U11 = scipy.linalg.expm(-1j*self.hams.H11(phi)*dt)
            psi01 = U01 @ psi01
            psi11 = U11 @ psi11
        # remove global phase
        gp = psi01[0]/np.abs(psi01[0])
        psi01 /= gp
        psi11 /= gp**2
        return self.fid.bell_state_fidelity(psi01, psi11)


class CRABOptimizer:
    """
    Wraps a CosineAnsatz and GateSimulator to tune ansatz parameters via SciPy.
    """
    def __init__(self,
                 ansatz: CosineAnsatz,
                 simulator: GateSimulator
                 ):
        self.ans   = ansatz
        self.sim   = simulator

    def loss(self, inputs: np.ndarray) -> float:
        phases, dt = self.ans.generate(self.sim.params, inputs)
        F = self.sim.run(phases, dt)
        return 1 - F

    def optimize(self, init_inputs: np.ndarray):
        def cb(xk):
            inf = self.loss(xk)
            print(f"Infidelity: {inf:.6f}", end="\r", flush=True)
        res = minimize(self.loss, init_inputs, callback=cb, options={'gtol':1e-6})
        return res.fun, res.x

In [20]:
if __name__ == "__main__":
    # set up
    params        = PulseParameters(2*np.pi*10, 2*np.pi*10, 10, 12e6, 0, 10, 200)
    hams          = Hamiltonians(params)
    fid_calc      = FidelityCalculator()
    ansatz        = CosineAnsatz(num_terms=4)
    simulator     = GateSimulator(params, hams, fid_calc)

    # initial guess: [pulse, drive_detuning, (A_i,f_i,o_i)] Amplitudes, Frequencies, phi_Offsets
    init = [
        3, 0,
        2*np.pi*0.1122, 1.043*params.Omega_Rabi1, -0.7318,
        #0.1,             1.043*params.Omega_Rabi1*2, -2.44,
        #0.01,            1.043*params.Omega_Rabi1*4, -5.15,
        #0.01,            1.043*params.Omega_Rabi1*8, -0.5
    ]

    optimizer = CRABOptimizer(ansatz, simulator)
    best_inf, best_x = optimizer.optimize(np.array(init))
    print("\nBest infidelity:", best_inf)
    print("Best parameters:", best_x)


Infidelity: 0.540307
Best infidelity: 0.540306921432624
Best parameters: [  5.81180142 191.83132486   6.80807622 122.72417107  -2.03994022]


In [25]:
class GRAPEProfile:
    """
    Handles loading a CRAB pulse and interpolating it onto a uniform GRAPE grid.
    """
    def __init__(self, params: PulseParameters):
        self.params = params

    def load(self, times_crab: np.ndarray, phi_crab: np.ndarray):
        self.times_crab = times_crab
        self.phi_crab   = phi_crab

    def interpolate(self, pulse_time: float, N: int):
        # uniform GRAPE grid
        self.N = N
        self.t_grape = np.linspace(0, pulse_time, N)
        self.u = np.interp(self.t_grape, self.times_crab, self.phi_crab)
        self.dt = pulse_time / N
        return self.u, self.dt


class GRAPEOptimizer:
    """
    Implements forward/backward propagation and gradient-based optimization (GRAPE).
    """
    def __init__(self,
                 profile: GRAPEProfile,
                 hams: Hamiltonians
                 ):
        self.profile = profile
        self.hams    = hams

    def forward_propagation(self, u: np.ndarray):
        N = self.profile.N
        psi01 = np.zeros((N+1, 3), complex)
        psi11 = np.zeros((N+1, 7), complex)
        psi01[0] = self.profile.params.initial_psi01.copy()
        psi11[0] = self.profile.params.initial_psi11.copy()
        for j in range(N):
            U01 = scipy.linalg.expm(-1j*self.hams.H01(u[j])*self.profile.dt)
            U11 = scipy.linalg.expm(-1j*self.hams.H11(u[j])*self.profile.dt)
            psi01[j+1] = U01 @ psi01[j]
            psi11[j+1] = U11 @ psi11[j]
        return psi01, psi11

    def backward_propagation(self, u: np.ndarray, psi01_f, psi11_f):
        N = self.profile.N
        S = 1 + 2*psi01_f[0] - psi11_f[0]
        chi01 = np.zeros((N+1, 3), complex)
        chi11 = np.zeros((N+1, 7), complex)
        chi01[N] = (np.conj(S)/8) * np.array([1,0,0], complex)
        chi11[N] = (-np.conj(S)/16) * np.array([1,0,0,0,0,0,0], complex)
        for j in reversed(range(N)):
            U01 = scipy.linalg.expm(-1j*self.hams.H01(u[j])*self.profile.dt)
            U11 = scipy.linalg.expm(-1j*self.hams.H11(u[j])*self.profile.dt)
            chi01[j] = U01.conj().T @ chi01[j+1]
            chi11[j] = U11.conj().T @ chi11[j+1]
        return chi01, chi11

    def fidelity_and_gradient(self, u: np.ndarray):
        psi01, psi11 = self.forward_propagation(u)
        psi01_f, psi11_f = psi01[-1], psi11[-1]
        S = 1 + 2*psi01_f[0] - psi11_f[0]
        F_bell = np.abs(S)**2 / 16
        chi01, chi11 = self.backward_propagation(u, psi01_f, psi11_f)
        grad = np.zeros_like(u)
        for j in range(self.profile.N):
            g01 = 2 * np.imag(chi01[j].conj().T @ (self.hams.dH01_dphi(u[j]) @ psi01[j])) * self.profile.dt
            g11 = 2 * np.imag(chi11[j].conj().T @ (self.hams.dH11_dphi(u[j]) @ psi11[j])) * self.profile.dt
            grad[j] = g01 + g11
        return 1 - F_bell, -grad

    def optimize(self, u_init: np.ndarray, maxiter=200):
        history = []
        def cb(xk):
            f, _ = self.fidelity_and_gradient(xk)
            history.append(1 - f)
            print(f"Infidelity: {1-f:.6f}", end="\r", flush=True)
        result = minimize(lambda x: self.fidelity_and_gradient(x)[0],
                          u_init,
                          jac=lambda x: self.fidelity_and_gradient(x)[1],
                          
                          options={'maxiter': maxiter,'gtol':1e-6},
                          callback=cb)
        return result.fun, result.x, history


In [26]:
#Run Grape
profile = GRAPEProfile(params)
# assume times_crab, phi_crab come from CRAB ansatz
times_crab = np.linspace(0, init[0], params.resolution)
phi_crab   = ansatz.generate(params, init)[0]
profile.load(times_crab, phi_crab)
u_init, dt = profile.interpolate(init[0], N=1000)
grape_opt = GRAPEOptimizer(profile, hams)
infid, u_opt, history = grape_opt.optimize(u_init)
print(f"\nOptimized GRAPE infidelity: {infid:.6f}")
print("Optimized GRAPE pulse u_opt:", u_opt)

Infidelity: 0.282994
Optimized GRAPE infidelity: 0.717006
Optimized GRAPE pulse u_opt: [ 0.73084279  0.7401256   0.74934359  0.75849582  0.76758137  0.77659938
  0.78548725  0.79430443  0.80305137  0.81172727  0.82033139  0.8288011
  0.8371948   0.84551439  0.85375922  0.86192866  0.86996009  0.87791077
  0.88578414  0.89357967  0.90129686  0.90887315  0.91636456  0.92377611
  0.9311074   0.93835809  0.9454657   0.95248492  0.95942241  0.96627791
  0.9730512   0.97967999  0.98621749  0.99267207  0.99904361  1.00533202
  1.01147524  1.01752488  1.02349108  1.02937386  1.03517327  1.04082753
  1.04638654  1.05186225  1.05725482  1.06256443  1.06772965  1.07279853
  1.0777849   1.08268906  1.0875113   1.09219061  1.09677306  1.10127442
  1.1056951   1.1100355   1.11423513  1.11833791  1.12236161  1.12630673
  1.13017382  1.13390292  1.13753568  1.14109193  1.14457226  1.14797732
  1.15124779  1.15442293  1.15752461  1.16055355  1.16351048  1.16633679
  1.16906918  1.17173166  1.17432504  